In [20]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms, models
from torch.utils.data import DataLoader
from torch.optim import Adam
import os
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# データの読み込み
data = pd.read_csv('banana_brix1.csv')  # CSVファイルに画像ファイル名とクラスラベルが含まれていると仮定

# 'image' カラムと 'judge' カラムだけを抽出
selected_data = data[['image', 'judge']]

# 抽出したデータを表示
selected_data

,image,judge
0,bbrix1.JPG,non-ripe
1,bbrix2-1.JPG,half-rotten
2,bbrix2-2.JPG,half-rotten
3,bbrix3.JPG,sweet
4,bbrix4-1.JPG,non-ripe
5,bbrix4-2.JPG,non-ripe
6,bbrix5-1.JPG,non-ripe
7,bbrix5-2.JPG,non-ripe
8,bbrix6-1.JPG,sweet
9,bbrxi6-2.JPG,sweet


In [11]:
import os
import joblib
from PIL import Image
import torchvision.transforms as T

# モデルを読み込む
model = joblib.load('bananaDetect2.pkl')
# 画像フォルダのパス
image_folder = 'banana_brix_images'
edit_image_folder = 'banana_brix_images_edit'
# デバイスの指定 (GPUが利用可能な場合はGPUを使用し、それ以外の場合はCPUを使用)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



# 1. 画像の前処理
def preprocess_image(image):
    transform = T.Compose([
        T.ToTensor(),  # 画像をテンソルに変換
    ])
    return transform(image).unsqueeze(0)  # バッチ次元を追加

# 2. モデルに画像を渡して推論
def predict(model, image_tensor):
    model.eval()
    with torch.no_grad():
        image_tensor = image_tensor.to(device)
        predictions = model(image_tensor)

    return predictions

# 画像フォルダ内の各画像に対して処理
for filename in os.listdir(image_folder):
    if filename.endswith('.JPG'):
        image_path = os.path.join(image_folder, filename)
        # 画像を開く
        pil_image = Image.open(image_path)
        # バウンディングボックスを検出
        image_tensor = preprocess_image(pil_image)
        predictions = predict(model, image_tensor)
        
        # しきい値を設定
        threshold = 0.5  # この値を必要に応じて調整
        # 最もスコアが高いバウンディングボックスを見つける
        best_score = 0.0
        best_box = None
        for score, label, box in zip(predictions[0]['scores'], predictions[0]['labels'], predictions[0]['boxes']):
            if score > threshold and score > best_score:
                best_score = score
                best_box = box

        if best_box is not None:
            best_box = [round(i, 2) for i in best_box.tolist()]
            x1, y1, x2, y2 = map(int, best_box)
            cropped_image = pil_image.crop((x1, y1, x2, y2))
            # 画像を保存
            edit_image_path = os.path.join(edit_image_folder, filename)
            cropped_image.save(edit_image_path)

print("編集済みの画像を保存しました。")


編集済みの画像を保存しました。


In [26]:
# クラスラベルを整数にエンコードする辞書を作成
label_encoding = {
    'non-ripe': 0,
    'sweet': 1,
    'half-rotten': 2,
    'rotten': 3
}

# データフレームの'judge'列を整数にエンコードして新しい'label'列を作成
data['label'] = data['judge'].map(label_encoding)

# 'label'列が正しく追加されたことを確認
data


,id,image,brix,check,judge,label
0,1,bbrix1.JPG,17.0,NaN,non-ripe,0
1,2,bbrix2-1.JPG,20.0,NaN,half-rotten,2
2,3,bbrix2-2.JPG,20.0,NaN,half-rotten,2
3,4,bbrix3.JPG,22.0,NaN,sweet,1
4,5,bbrix4-1.JPG,17.0,NaN,non-ripe,0
5,6,bbrix4-2.JPG,17.0,NaN,non-ripe,0
6,7,bbrix5-1.JPG,19.0,NaN,non-ripe,0
7,8,bbrix5-2.JPG,19.0,NaN,non-ripe,0
8,9,bbrix6-1.JPG,21.0,NaN,sweet,1
9,10,bbrxi6-2.JPG,21.0,NaN,sweet,1


In [36]:
from torch.utils.data import Dataset
import torch.optim as optim


# 画像フォルダのパス
image_folder = 'banana_brix_images_edit'  # 保存した編集済み画像のフォルダ

# 画像の前処理
transform = T.Compose([
    T.Resize((224, 224)),  # 画像を指定のサイズにリサイズ
    T.ToTensor(),  # 画像をテンソルに変換
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # 画像を標準化
])




class CustomDataset(Dataset):
    def __init__(self, dataframe, image_folder, transform=None):
        self.dataframe = dataframe
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_folder, str(self.dataframe.loc[idx, 'image']))  # 'image'列を文字列に変換
        # ファイルの存在確認
        if os.path.exists(img_name):
            image = Image.open(img_name)
            label = int(self.dataframe.loc[idx, 'label'])  # 'label'列を使用
        else:
            # ファイルが存在しない場合の処理を記述
            print(f"File not found: {img_name}")
            # 例外をスローする場合： raise FileNotFoundError(f"File not found: {img_name}")
            # デフォルト画像を使用する場合： image = Image.open('default_image.jpg')
            return self.__getitem__((idx + 1) % len(self.dataframe))  # 次のデータを再帰的に読み込む

        if self.transform:
            image = self.transform(image)

        return image, label



    
    
    
# データセットをトレーニングデータとテストデータに分割
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# カスタムデータセットを作成
train_dataset = CustomDataset(train_data, image_folder, transform=transform)
test_dataset = CustomDataset(test_data, image_folder, transform=transform)

# データローダーを作成
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# ニューラルネットワークモデルの定義 (例: ResNet)
model = models.resnet18(pretrained=True)
num_classes = 4  # クラス数を指定

# モデルの最終層を変更して、出力クラス数に合わせる
model.fc = nn.Linear(model.fc.in_features, num_classes)

# 損失関数とオプティマイザを定義
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# モデルの訓練
num_epochs = 10  # エポック数を指定 (必要に応じて調整)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        
        # 画像の前処理を行わずに、前処理済みの画像データを使用
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # エポックごとの損失を表示
    print(f'Epoch [{epoch + 1}/{num_epochs}] Loss: {running_loss / len(train_loader)}')

print('Finished Training')




KeyError: 34

In [38]:
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torchvision.transforms as T
from sklearn.model_selection import train_test_split
from torchvision import models
import torch.nn as nn
from PIL import Image
import os
import pandas as pd

# 画像フォルダのパス
image_folder = 'banana_brix_images_edit'  # 保存した編集済み画像のフォルダ

# 画像の前処理
transform = T.Compose([
    T.Resize((224, 224)),  # 画像を指定のサイズにリサイズ
    T.ToTensor(),  # 画像をテンソルに変換
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # 画像を標準化
])

class CustomDataset(Dataset):
    def __init__(self, dataframe, image_folder, transform=None):
        self.dataframe = dataframe
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_folder, str(self.dataframe.iloc[idx]['image']))  # 'image'列を文字列に変換
        # ファイルの存在確認
        if os.path.exists(img_name):
            image = Image.open(img_name)
            label = int(self.dataframe.iloc[idx]['label'])  # 'label'列を使用
        else:
            # ファイルが存在しない場合の処理を記述
            print(f"File not found: {img_name}")
            # 例外をスローする場合： raise FileNotFoundError(f"File not found: {img_name}")
            # デフォルト画像を使用する場合： image = Image.open('default_image.jpg')
            return None, None  # ダミーの値を返すことでスキップします

        if self.transform:
            image = self.transform(image)

        return image, label

# データセットをトレーニングデータとテストデータに分割
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# カスタムデータセットを作成
train_dataset = CustomDataset(train_data, image_folder, transform=transform)
test_dataset = CustomDataset(test_data, image_folder, transform=transform)

# データローダーを作成
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# ニューラルネットワークモデルの定義 (例: ResNet)
model = models.resnet18(pretrained=True)
num_classes = 4  # クラス数を指定

# モデルの最終層を変更して、出力クラス数に合わせる
model.fc = nn.Linear(model.fc.in_features, num_classes)

# 損失関数とオプティマイザを定義
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# モデルの訓練
num_epochs = 10  # エポック数を指定 (必要に応じて調整)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # エポックごとの損失を表示
    print(f'Epoch [{epoch + 1}/{num_epochs}] Loss: {running_loss / len(train_loader)}')

print('Finished Training')


Epoch [1/10] Loss: 1.2976568341255188
Epoch [2/10] Loss: 0.17372682690620422
Epoch [3/10] Loss: 1.0455231070518494
Epoch [4/10] Loss: 1.2014157325029373
Epoch [5/10] Loss: 0.9426499381661415
Epoch [6/10] Loss: 0.3922082707285881
Epoch [7/10] Loss: 1.5479343086481094
Epoch [8/10] Loss: 1.1720211505889893
Epoch [9/10] Loss: 0.5793784707784653
Epoch [10/10] Loss: 1.2011470198631287
Finished Training


In [46]:
from PIL import Image
import torch
import torchvision.transforms as T


# 1. モデルを読み込む
model_d = joblib.load('bananaDetect2.pkl')

# 2. 画像の前処理
def preprocess_image(image):
    transform = T.Compose([
        T.ToTensor(),  # 画像をテンソルに変換
    ])
    return transform(image).unsqueeze(0)  # バッチ次元を追加

# 3. モデルに画像を渡してバウンディングボックスを検出
def detect_bounding_box(model, image_tensor, threshold=0.5):
    model.eval()
    with torch.no_grad():
        image_tensor = image_tensor.to(device)
        predictions = model(image_tensor)

    # しきい値を超える最もスコアの高いバウンディングボックスを見つける
    best_score = 0.0
    best_box = None
    for score, label, box in zip(predictions[0]['scores'], predictions[0]['labels'], predictions[0]['boxes']):
        if score > threshold and score > best_score:
            best_score = score
            best_box = box

    return best_box


# 4. 切り取った画像に対して分類を行う
def classify_banana(model, image):
    # 画像の前処理を適用
    image_tensor = preprocess_image(image)
    
    # モデルに画像を入力し、予測を取得
    with torch.no_grad():
        outputs = model(image_tensor)
    
    # 予測クラスの取得
    _, predicted_class = outputs.max(1)
    
    return predicted_class.item()

# 画像フォルダ内の各画像に対して処理
image_folder = 'bananas_images_test'  # 画像フォルダのパスを指定

for filename in os.listdir(image_folder):
    if filename.endswith('.JPG'):
        image_path = os.path.join(image_folder, filename)
        # 画像を開く
        pil_image = Image.open(image_path)
        # バウンディングボックスを検出
        image_tensor = preprocess_image(pil_image)
        bounding_box = detect_bounding_box(model_d, image_tensor)
        
        if bounding_box is not None:
            # バウンディングボックス情報を取得
            x1, y1, x2, y2 = map(int, bounding_box.tolist())
            cropped_image = pil_image.crop((x1, y1, x2, y2))
            
            # 分類を実行
            predicted_class = classify_banana(model, cropped_image)
            
            # 予測クラスを表示
            print(f"Predicted class for {filename}: {predicted_class}")
            

Predicted class for b4.JPG: 1
Predicted class for b3.JPG: 1
Predicted class for b2.JPG: 1
Predicted class for b1.JPG: 1


In [47]:
# モデルをPickleファイルに保存
model_path = 'bananaClassify.pkl'
joblib.dump(model, model_path)

['bananaClassify.pkl']